# Load options data files
I've downloaded the options data for the `tickers` you requested for both `call` and `put` contracts for 45-day contracts starting on `2023-01-03`. Let's walk through the `call` contract data (the `put` contract procedure is the same; we change the `jld2` file name to `Options-Put-Daily-OHLC-45d-2023.jld2`)

## Setup

In [1]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Manifest.toml`
  Activating project at `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Documents/GitHub/CHEME5660/CHEME-5660-Project-Template-F23/Manifest.toml`


log_return_matrix (generic function with 1 method)

## Load `call` contract data
We downloaded data for 45-day contracts starting on `2023-01-03` and running through `2023`. The date information for each contract is given in the `call_contract_dates` variable. This is a `Tuple` that holds the day the contract was sold (`index = 1`), the day the contract expires (`index = 2`), and the number of days of the contract (`index=3`):

In [2]:
call_contract_dates = load(joinpath(_PATH_TO_DATA, "Options-Call-Daily-OHLC-45d-2023.jld2")) |> x -> x["dates"]

8-element Vector{Tuple{Date, Date, Int64}}:
 (Date("2023-01-03"), Date("2023-02-17"), 45)
 (Date("2023-02-22"), Date("2023-04-14"), 51)
 (Date("2023-04-17"), Date("2023-06-02"), 46)
 (Date("2023-06-05"), Date("2023-07-21"), 46)
 (Date("2023-07-24"), Date("2023-09-08"), 46)
 (Date("2023-09-11"), Date("2023-10-27"), 46)
 (Date("2023-10-30"), Date("2023-12-15"), 46)
 (Date("2023-12-18"), Date("2024-02-02"), 46)

To see the price information for each `call` contract, we load the `call_contract_data` dictionary. This data structure holds the price values for each ticker and each date combination. 

In [3]:
call_contract_data = load(joinpath(_PATH_TO_DATA, "Options-Call-Daily-OHLC-45d-2023.jld2")) |> x -> x["dataset"]

Dict{Tuple, NamedTuple} with 56 entries:
  ("AMD", Date("2023-04-17… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-02-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-06-05… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-10-3… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-01-03… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AMD", Date("2023-02-22… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, 

We access this information by constructing a `tuple,` with the first item being the `ticker,` the second being the `sell` data, and the third item being the expiration date.
* For example, to look at the data for an `AMD` call sold on `2023-06-05` and expiring on `2023-07-21`, the `key` would be:

In [1]:
test_key = ("MSFT", Date("2023-01-03"), Date("2023-02-17"));

LoadError: UndefVarError: `Date` not defined

passing this `key` into the `call_contract_data` dictionary gives us a `dictionary` holding the `OHLC` data for several strike prices:

In [5]:
MSFT = call_contract_data[test_key] |> x-> x[:data]

Dict{Float64, Union{Nothing, DataFrame}} with 13 entries:
  260.0 => 33×8 DataFrame…
  230.0 => 33×8 DataFrame…
  265.0 => 33×8 DataFrame…
  210.0 => 29×8 DataFrame…
  220.0 => 33×8 DataFrame…
  255.0 => 33×8 DataFrame…
  225.0 => 33×8 DataFrame…
  245.0 => 33×8 DataFrame…
  215.0 => 26×8 DataFrame…
  240.0 => 33×8 DataFrame…
  235.0 => 33×8 DataFrame…
  250.0 => 33×8 DataFrame…
  270.0 => 33×8 DataFrame…

In [103]:
MSFT[260.0][1,:]

Row,volume,volume_weighted_average_price,open,close,high,low,timestamp,number_of_transactions
,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Int64
1,618.0,4.7143,5.32,4.1,6.1,3.85,2023-01-03T05:00:00,121


In [105]:
MSFT[260.0][1,:2]*100

471.42999999999995

In [8]:
original_dataset = load(joinpath(_PATH_TO_DATA, 
        "SP500-Daily-OHLC-1-3-2018-to-12-01-2023.jld2")) |> x-> x["dataset"];

In [35]:
maximum_number_trading_days = original_dataset["MSFT"] |> nrow;

dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset;

all_tickers = keys(dataset) |> collect |> sort;
K = length(all_tickers);

startdate = Date(2023,01,03);
MSFT_dataset = dataset["MSFT"];
MSFT_df = filter(:timestamp => x-> x >= startdate, MSFT_dataset);

In [36]:
MSFT_df[MSFT_df.timestamp .== DateTime(2023, 1, 3, 5, 0, 0), :]

Row,volume,volume_weighted_average_price,open,close,high,low,timestamp,number_of_transactions
,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Int64
1,2.574e7,239.839,243.08,239.58,245.75,237.4,2023-01-03T05:00:00,314904


In [37]:
MSFT_df[MSFT_df.timestamp .== DateTime(2023, 2, 17, 5, 0, 0), :]

Row,volume,volume_weighted_average_price,open,close,high,low,timestamp,number_of_transactions
,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Int64
1,2.99982e7,257.685,259.39,258.06,260.09,256.0,2023-02-17T05:00:00,392723


In [111]:
function option_value(market_df::DataFrame, options_df::DataFrame, strike_price::Float64)
    last_day = maximum(options_df[:, :timestamp])
    last_day_market_info = market_df[market_df[:, :timestamp] .== last_day, :]

    last_day_market_price = last_day_market_info[1, :close]
    if last_day_market_price < strike_price
        # If the option expires worthless return 1.0, otherwise return 0.0
        return 1.0
    else
        return 0.0
    end
end


option_value (generic function with 1 method)

In [112]:
for (strike_price, options_df) in MSFT
    result = option_value(MSFT_df, options_df, strike_price)
        println("For a MSFT short call with a strike price $strike_price: $result")
end

For a MSFT short call with a strike price 260.0: 1.0
For a MSFT short call with a strike price 230.0: 0.0
For a MSFT short call with a strike price 265.0: 1.0
For a MSFT short call with a strike price 210.0: 0.0
For a MSFT short call with a strike price 220.0: 0.0
For a MSFT short call with a strike price 255.0: 0.0
For a MSFT short call with a strike price 225.0: 0.0
For a MSFT short call with a strike price 245.0: 0.0
For a MSFT short call with a strike price 215.0: 0.0
For a MSFT short call with a strike price 240.0: 0.0
For a MSFT short call with a strike price 235.0: 0.0
For a MSFT short call with a strike price 250.0: 0.0
For a MSFT short call with a strike price 270.0: 1.0


In [110]:
for (strike_price, options_df) in MSFT
    premium = round(options_df[1,:2]*100, digits=2)
        println("For a MSFT short call with a strike price $strike_price: The premium will be $premium")
end

For a MSFT short call with a strike price 260.0: The premium will be 471.43
For a MSFT short call with a strike price 230.0: The premium will be 1959.0
For a MSFT short call with a strike price 265.0: The premium will be 308.08
For a MSFT short call with a strike price 210.0: The premium will be 2391.28
For a MSFT short call with a strike price 220.0: The premium will be 2430.0
For a MSFT short call with a strike price 255.0: The premium will be 671.13
For a MSFT short call with a strike price 225.0: The premium will be 2057.0
For a MSFT short call with a strike price 245.0: The premium will be 1100.54
For a MSFT short call with a strike price 215.0: The premium will be 2821.0
For a MSFT short call with a strike price 240.0: The premium will be 1200.45
For a MSFT short call with a strike price 235.0: The premium will be 1472.3
For a MSFT short call with a strike price 250.0: The premium will be 782.83
For a MSFT short call with a strike price 270.0: The premium will be 262.69


In [113]:
call_contract_data

Dict{Tuple, NamedTuple} with 56 entries:
  ("AMD", Date("2023-04-17… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-02-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-06-05… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-10-3… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-01-03… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("MSFT", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AMD", Date("2023-02-22… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, 

In [116]:
tech_portfolio = ["AMD", "AAPL", "MSFT", "NVDA", "ADBE", "SPY"]
tech_contracts = Dict()

for (key, value) in call_contract_data
    ticker, date = key
    if ticker in tech_portfolio
        tech_contracts[key] = value
    end
end

In [117]:
tech_contracts

Dict{Any, Any} with 42 entries:
  ("ADBE", Date("2023-02-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-10-3… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AMD", Date("2023-04-17… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-02-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-06-05… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-07-2… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("AAPL", Date("2023-10-3… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-09-11… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("ADBE", Date("2023-06-0… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("SPY", Date("2023-01-03… => (data = Dict{Float64, Union{Nothing, DataFrame}}…
  ("NVDA", Date("2023-04-1… => (data = Dict{Float64, Union{Nothing, DataFrame

In [133]:
call_contract_dates[1:3]

3-element Vector{Tuple{Date, Date, Int64}}:
 (Date("2023-01-03"), Date("2023-02-17"), 45)
 (Date("2023-02-22"), Date("2023-04-14"), 51)
 (Date("2023-04-17"), Date("2023-06-02"), 46)